In [1]:
from bs4 import BeautifulSoup
import grequests
import requests
from multiprocessing import Pool
import numpy as np

In [2]:
def strip_white_chars(text):
    return text.strip("\n").strip(" ")

In [97]:
def get_product_data(product_request):
    """Function is responsible for scraping product data from 4U.am product content
    """
    soup = BeautifulSoup(product_request.content, 'html.parser')
    sp = soup.find_all("div", {"class": "info_container"})[0]
    
    product_data = {}
    product_data["product"] = sp.find('p', attrs={'class' : 'name_info'}).text
    
    brand_partner = sp.find_all('div', attrs={'class' : 'brand_partner'})
    product_data["item_code"] = brand_partner[0].span.strong.text
    product_data["store"] = strip_white_chars(brand_partner[1].find_all("span")[0].a.text)
    try:
        product_data["brand"] = strip_white_chars(brand_partner[1].find_all("span")[1].a.text)
    except:
        pass
    product_data["url"] = product_request.url
    product_data["price"] = strip_white_chars(sp.find("p", attrs={'class' : 'item_price'}).span.span.text)
    product_data["item description"] = strip_white_chars(sp.find("div", attrs={'id' : 'description'}).div.text)
    product_data["image url"] = soup.find("div", attrs={'class' : 'show_tumb'}).img["src"]
    
    return product_data

In [98]:
def extract_category_contents(category_names):
    rs = (grequests.get("https://4u.am/hy/category/"+cat) for cat in category_names)
    return grequests.map(rs)

def extract_item_contents(item_urls):
    rs = (grequests.get("https://4u.am"+item) for item in item_urls)
    return grequests.map(rs)

In [99]:
def extract_item_urls(category_request):
    items = []
    soup = BeautifulSoup(category_request.content, 'html.parser')
    sp= soup.find_all('div', attrs={'class' : 'category_item'})
    for i in sp: items.append(i.find_all("a")[1]["href"])
    return items

In [100]:
def extract_item_info(category_names):
    
    with Pool(5) as p:
        item_urls = p.map(extract_item_urls, extract_category_contents(category_names), chunksize=1)
    urls = [j for i in item_urls for j in i]

    with Pool(5) as p:
        result = p.map(get_product_data,extract_item_contents(urls))
    
    return result

In [101]:
test = extract_item_info(category_urls)

In [106]:
import pickle
with open("data.pkl", "wb") as f:
    pickle.dump(test,f)

In [111]:
with open("data.pkl", "rb") as f:
    qq = pickle.load(f)

In [112]:
qq

[{'product': 'Մատանի «DF Project» արծաթյա №14',
  'item_code': 'N:051649',
  'store': 'DF project',
  'brand': 'DF Project',
  'url': 'https://4u.am/hy/product/matani-df-project-arcatya-no14',
  'price': '9,000',
  'item description': '925 հարգի արծաթյա մատանի:\nՉափը՝ կարգավորվող:\nՔարը՝ օնիքս։\nՔաշը՝ 3.5 գ։',
  'image url': 'https://static.4u.am/origin/product/1024/dejhlp-iRIAG78l4.jpg'},
 {'product': 'Բրոշ «LilmArt» աստղ',
  'item_code': 'N:006014',
  'store': 'LilmArt',
  'url': 'https://4u.am/hy/product/bros-lilmart-astg',
  'price': '4,400',
  'item description': 'Բրոշը ամբոջությամբ ձեռքի աշխատանք է:\n\n\nՔարերը`բյուրեղ:\n\n\nԵրկարությունը` 4 սմ: \n\n\nԿարող եք պատվիրել Ձեր ցանկացած չափի:',
  'image url': 'https://static.4u.am/origin/product/1024/0sTXJB1jHm82-zwM.jpg'},
 {'product': 'Թևնոց «SOKOLOV» 94050369',
  'item_code': 'N:074276',
  'store': 'Charm.am',
  'url': 'https://4u.am/hy/product/tevnoc-sokolov-94050369',
  'price': '13,900',
  'item description': 'Ապրանքանիշ` SOKOLO